In [1]:
import os
print(os.getcwd())
def update_working_directory():
    from pathlib import Path
    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)
update_working_directory()

/Users/admin/Projects/doggo/notebooks
/Users/admin/Projects/doggo


# Importing the required libraries

In [2]:
import numpy as np
import pandas as pd
import gym
import time
import math
import statistics
from tqdm import tqdm
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import dill

# Building the environment

## Parameters specific to the environment

In [3]:
decimals_state = 2
gamma = 0.95 # discount for future rewards (also called decay factor)

## Observation & Action spaces

In [4]:
n_states = (10**decimals_state+1)**3 

action_dict = {
    0: 'NO ACTION',
    1: 'WALKING',
    2: 'EATING',
    3: 'PLAYING'
}
n_actions= len(action_dict)

print(n_states, n_actions)

1030301 4


## Reset

In [5]:
def get_state_id(dog_state):
    return '{:01.4f}_{:01.4f}_{:01.4f}_{}'.format(
        dog_state['food'], dog_state['fat'], dog_state['affection'], dog_state['can_action_be_taken'])

In [6]:
def env_reset():
    
    dog_state = {
        'food': 0.5,
        'fat': 0,
        'affection': 0.5,
        'last_action_taken': 0,
        'minutes_since_last_action': 0,
        'can_action_be_taken': True
        }
    
    dog_state['state_id'] = get_state_id(dog_state)
    
    return dog_state

In [7]:
env_reset()

{'food': 0.5,
 'fat': 0,
 'affection': 0.5,
 'last_action_taken': 0,
 'minutes_since_last_action': 0,
 'can_action_be_taken': True,
 'state_id': '0.5000_0.0000_0.5000_True'}

## Next state

In [8]:
WALKING_TIME = 15
EATING_TIME = 1
PLAYING_TIME = 4

food_consumption_rate = 1.0 / (30 * 3600)
affection_consumption_rate = 1.0 / (50 * 3600)
walking_fat_converge_rate = 0.2
walking_affection_converge_rate = 0.4
playing_fat_converge_rate = 0.1
playing_affection_converge_rate = 0.20
eating_food_increase = 0.6
eating_fat_increase = 0.25

In [9]:
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier
def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

In [10]:
def apply_decreasing_rate(value: float, rate: float) -> float:
    """
    Apply a decreasing rate to a value
    :param value: current value
    :param rate: per second
    :return: updated value
    """
    return value - (60 * rate)

def converge(value: float, target: float, ratio: float) -> float:
    diff: float = (target - value) * ratio
    return value + diff


def update_food(dog_state):
    update_food = apply_decreasing_rate(dog_state['food'], food_consumption_rate)
    return round_down(max(0.0, update_food), decimals=decimals_state)

def update_fat(dog_state):
    update_fat = dog_state['fat']
    return update_fat

def update_affection(dog_state):
    update_affection = apply_decreasing_rate(dog_state['affection'], affection_consumption_rate)
    return round_down(max(0.0, update_affection), decimals=decimals_state)


def update_if_walking(dog_state):
    update_fat = round_down(converge(dog_state['fat'], 0.0, walking_fat_converge_rate), decimals=decimals_state)
    update_affection = round_up(converge(dog_state['affection'], 1.0, walking_affection_converge_rate), decimals=decimals_state)
    return (update_fat, update_affection)

def update_if_feeding(dog_state):
    update_food = round_up(min(dog_state['food'] + eating_food_increase, 1.0), decimals=decimals_state)
    update_fat = round_up(min(dog_state['fat'] + eating_fat_increase, 1.0), decimals=decimals_state)
    return (update_food, update_fat)

def update_if_playing(dog_state):
    update_fat = round_down(converge(dog_state['fat'], 0.0, playing_fat_converge_rate), decimals=decimals_state)
    update_affection = round_up(converge(dog_state['affection'], 1.0, playing_affection_converge_rate), decimals=decimals_state)
    return (update_fat, update_affection)


def get_happiness(dog_state):
    happiness = min(dog_state['food'], 1.0 - dog_state['fat'], dog_state['affection'])
    return happiness


def update_done(dog_state):
    happiness = get_happiness(dog_state)
    return happiness <= 0.0

In [11]:
# state2, reward1, done, info = env.step(action1)
def env_step(state1, action):
    
    state2 = state1.copy()
    reward_penalty = 0
    
    # Affect of time
    state2['food'] = update_food(state2)
    state2['fat'] = update_fat(state2)
    state2['affection'] = update_affection(state2)
    state2['minutes_since_last_action'] += 1 
    
    # Applying action
    if action != 0:
        if state2['can_action_be_taken']:
            reward_penalty += 0.1
            state2['can_action_be_taken'] = False
            state2['minutes_since_last_action'] = 0
            state2['last_action_taken'] = action
        else:
            reward_penalty += 0.5

    # Affect of actions
    if (state2['last_action_taken'] == 1) & (state2['minutes_since_last_action'] == WALKING_TIME):
        state2['fat'], state2['affection'] = update_if_walking(state2)
        state2['can_action_be_taken'] = True

    if (state2['last_action_taken'] == 2) & (state2['minutes_since_last_action'] == EATING_TIME):
        state2['food'], state2['fat'] = update_if_feeding(state2)
        state2['can_action_be_taken'] = True

    if (state2['last_action_taken'] == 3) & (state2['minutes_since_last_action'] == PLAYING_TIME):
        state2['fat'], state2['affection'] = update_if_playing(state2)
        state2['can_action_be_taken'] = True
                    
    done = update_done(state2)
    if done:
        reward = -10
    else:
        reward = min(state2['food'], 1.0 - state2['fat'], state2['affection']) - reward_penalty
    
    info = None
    
    state2['state_id'] = get_state_id(state2)
    
    return (state2, reward, done, info)

## Render

In [12]:
def env_render(dog_state, action, Q):
    print(dog_state)
    print(action)
    print(Q[dog_state['state_id']])

# Defining utility functions to be used in the learning process

## Initialising Q

In [13]:
def init_Q(n_actions, init_Q_type="ones"):
    """
    @param n_actions the number of actions
    @param type random, ones or zeros for the initialization
    """
    if init_Q_type == "ones":
        default_Q_values = np.ones(n_actions)
    elif init_Q_type == "random":
        default_Q_values = np.random.random(n_actions)
    elif init_Q_type == "zeros":
        default_Q_values = np.zeros(n_actions)
    
    def get_default_Q_values():
        return default_Q_values

    return defaultdict(get_default_Q_values)

## Initialising N

In [14]:
def init_N(n_actions):
    """
    @param n_actions the number of actions
    """
    default_N_values = np.zeros(n_actions)
    
    def get_default_N_values():
        return default_N_values

    return defaultdict(get_default_N_values)

## Choose an action

In [15]:
# Numpy generator
rng = np.random.default_rng()  # Create a default Generator.

In [16]:
def select_best_action(Q_state):
    winner = np.argwhere(Q_state == np.amax(Q_state))
    winner_list = winner.flatten().tolist()
    action = random.choice(winner_list)
    return action

### $\epsilon$-Greedy

In [17]:
def epsilon_greedy(Q, state_id, n_actions, epsilon):
    """
    @param Q Q values {state, action} -> value
    @param epsilon for exploration
    @param n_actions number of actions
    @param state state at time t
    """
    if rng.uniform(0, 1) < epsilon:
        action = np.random.randint(0, n_actions)
    else:
        action = select_best_action(Q[state_id])
    
    return action

## Discounted reward

In [18]:
def get_discounted_reward(t, l_rewards_episode, gamma):
    l_discounted_reward_episode = [t_prime_reward*(gamma**t_prime) for (t_prime, t_prime_reward) in enumerate(l_rewards_episode[t:])]
    G_k_t = sum(l_discounted_reward_episode)
    return G_k_t

def add_discounted_reward(steps_episode, gamma):
    l_rewards_episode = [step_episode['reward'] for step_episode in steps_episode]
    for (t, step_episode) in enumerate(steps_episode):
        step_episode['discounted_reward'] = get_discounted_reward(t, l_rewards_episode, gamma)

## Update N-matrice

In [19]:
def update_N_MC(N, step_episode, method_monte_carlo, states_already_visited=[]):
    
    state_id = step_episode['state']['state_id']
    action = step_episode['action']
    
    previous_N_value_state = N[state_id].copy()
    
    if method_monte_carlo == 'first_visit':
        if not state_id in states_already_visited:
            new_N_value = N[state_id][action] + 1
            previous_N_value_state[action] = new_N_value
    
    if method_monte_carlo == 'every_visit':
        new_N_value = N[state_id][action] + 1
        previous_N_value_state[action] = new_N_value

    N[state_id] = previous_N_value_state

## Update Q-matrice (state-action value function)

### Monte-Carlo

In [20]:
def update_Q_MC(Q, N, step_episode, method_monte_carlo, states_already_visited=[]):
    
    state_id = step_episode['state']['state_id']
    action = step_episode['action']
    G_k_t = step_episode['discounted_reward']
    
    previous_Q_value_state = Q[state_id].copy()
    
    if method_monte_carlo == 'first_visit':
        if not state_id in states_already_visited:
            new_Q_value = Q[state_id][action] + (G_k_t - Q[state_id][action]) / N[state_id][action]
            previous_Q_value_state[action] = new_Q_value
    
    if method_monte_carlo == 'every_visit':
        new_Q_value = Q[state_id][action] + (G_k_t - Q[state_id][action]) / N[state_id][action]
        previous_Q_value_state[action] = new_Q_value

    Q[state_id] = previous_Q_value_state

### SARSA

In [21]:
# Function to learn the Q-value  - Is it temporal-difference?
def update_Q_SARSA(Q, state1_id, action1, reward1, state2_id, action2, expected=False):
    
    previous_Q_value_state1 = Q[state1_id].copy()
    
    predict = Q[state1_id][action1] 
    
    target = reward1 + gamma * Q[state2_id][action2] 
    if expected:
        expected_value = np.mean(Q[state2_id])
        target = reward1 + gamma * expected_value
    
    new_Q_value = Q[state1_id][action1] + alpha * (target - predict)
    previous_Q_value_state1[action1] = new_Q_value
    
    Q[state1_id] = previous_Q_value_state1
        
    return Q

### Q-learning

In [22]:
# Function to learn the Q-value 
def update_Q_Qlearning(Q, state1_id, action1, reward1, state2_id, action2, expected=False):
    
    previous_Q_value_state1 = Q[state1_id].copy()
    
    predict = Q[state1_id][action1] 
    
    target = reward1 + gamma * Q[state2_id][action2] 
    
    new_Q_value = Q[state1_id][action1] + alpha * (target - predict)
    previous_Q_value_state1[action1] = new_Q_value
    
    Q[state1_id] = previous_Q_value_state1
    
    return Q

## Updating parameters

### Epsilon $\epsilon$ - Exploration rate

In [23]:
def get_epsilon(episode, init_epsilon):
    
    n_epsilon = init_epsilon/(episode+1)
    
    return n_epsilon

### Alpha $\alpha$ - Learning rate

In [24]:
def get_alpha(episode, init_alpha):
    
    n_alpha = init_alpha/(episode+1)
    
    return n_alpha

## Plots Reward / Steps / Happiness

In [25]:
import matplotlib.colors as colors

def get_list_colors():
    colors_list = ['r','g','b','k','darkorange','y','lime','c','m'] + list(colors._colors_full_map.values())
    return colors_list

colors = get_list_colors()

In [26]:
import numpy as np
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

### Reward

In [27]:
import numpy as np
import matplotlib.pyplot as plt

def plot_evolution_reward(evolution_reward, method_name):
    
    n_moving_points = int(np.ceil(len(evolution_reward)/100))
    y = running_mean(evolution_reward,n_moving_points)
    x = range(len(y))

    info_parameters = '{} method - {} steps \n {} init_epsilon - {} init_alpha - {} gamma - {} nmax_steps'.format(
        method_name, len(evolution_reward), init_epsilon, init_alpha, gamma, nmax_steps)
    
    plt.plot(x, y)
    plt.title('Evolution of Avg Reward per step per episode over time \n (smoothed over window size {})'.format(n_moving_points))
    plt.xlabel('Episode \n '+ info_parameters)
    plt.ylabel('Avg Reward per step per episode (Smoothed)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('data/figures/{}__reward.png'.format(method_name), format='png', dpi=500)
    plt.show()

In [28]:
def plot_comparison_evolution_reward(evo_training__evo_avg_reward_per_step):

    n_episodes = len(list(evo_training__evo_avg_reward_per_step.values())[0])
    info_parameters = 'All methods - {} episodes \n {} init_epsilon - {} init_alpha - {} gamma - {} nmax_steps'.format(
        n_episodes, init_epsilon, init_alpha, gamma, nmax_steps)

    fig = plt.figure()
    cnt=0
    for method in list(evo_training__evo_avg_reward_per_step.keys()):

        n_moving_points = int(np.ceil(len(evo_training__evo_avg_reward_per_step[method])/100))
        y = running_mean(evo_training__evo_avg_reward_per_step[method], n_moving_points)
        x = range(len(y))

        plt.plot(
            x
            , y
            , label=method
            , marker='', color=colors[cnt], linewidth=1, alpha=0.75
        )
        cnt += 1

    plt.title('Evolution of Avg Reward per step per episode over time \n (smoothed over window size {})'.format(n_moving_points))
    plt.xlabel('Episode \n '+ info_parameters)
    plt.ylabel('Avg Reward per step \n per episode (Smoothed)')
    plt.legend(bbox_to_anchor=(0.5,-0.10), loc="lower center", 
                bbox_transform=fig.transFigure, ncol=4, fancybox=True, shadow=True, borderpad=1)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('data/figures/Comparison__reward.png', format='png', dpi=500)
    plt.show()

### Number of steps

In [29]:
import numpy as np
import matplotlib.pyplot as plt

def plot_evolution_steps(evolution_steps, method_name):
    
    n_moving_points = int(np.ceil(len(evolution_steps)/100))
    y = running_mean(evolution_steps,n_moving_points)
    x = range(len(y))

    info_parameters = '{} method - {} steps \n {} init_epsilon - {} init_alpha - {} gamma - {} nmax_steps'.format(
        method_name, len(evolution_steps), init_epsilon, init_alpha, gamma, nmax_steps)
    
    plt.plot(x, y)
    plt.title('Episode Length over time \n (smoothed over window size {})'.format(n_moving_points))
    plt.axhline(nmax_steps, color = 'r')
    plt.axhline(0, color = 'b')
    plt.ylim([-10, nmax_steps*1.05])
    plt.xlabel('Episode \n '+ info_parameters)
    plt.ylabel('Episode Length (Smoothed)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('data/figures/{}__steps.png'.format(method_name), format='png', dpi=500)
    plt.show()

In [30]:
def plot_comparison_evolution_steps(evo_training__evo_n_steps):

    n_episodes = len(list(evo_training__evo_n_steps.values())[0])
    info_parameters = 'All methods - {} episodes \n {} init_epsilon - {} init_alpha - {} gamma - {} nmax_steps'.format(
        n_episodes, init_epsilon, init_alpha, gamma, nmax_steps)

    fig = plt.figure()
    cnt=0
    for method in list(evo_training__evo_n_steps.keys()):

        n_moving_points = int(np.ceil(len(evo_training__evo_n_steps[method])/100))
        y = running_mean(evo_training__evo_n_steps[method], n_moving_points)
        x = range(len(y))

        plt.plot(
            x, y, label=method
            , marker='', color=colors[cnt], linewidth=1, alpha=0.75
        )
        cnt += 1

    plt.title('Episode Length over time \n (smoothed over window size {})'.format(n_moving_points))
    plt.axhline(nmax_steps, color = 'r')
    plt.axhline(0, color = 'b')
    plt.ylim([-10, nmax_steps*1.05])
    plt.xlabel('Episode \n '+ info_parameters)
    plt.ylabel('Episode Length (Smoothed)')
    plt.legend(bbox_to_anchor=(0.5,-0.10), loc="lower center", 
            bbox_transform=fig.transFigure, ncol=4, fancybox=True, shadow=True, borderpad=1)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('data/figures/Comparison__steps.png', format='png', dpi=500)
    plt.show()

### Happiness

In [31]:
import numpy as np
import matplotlib.pyplot as plt

def plot_evolution_happiness(evolution_happiness_all, method_name):
    
    n_moving_points = int(np.ceil(len(evolution_happiness_all)/100))
    y = running_mean(evolution_happiness_all,n_moving_points)
    x = range(len(y))

    info_parameters = '{} method - {} steps \n {} init_epsilon - {} init_alpha - {} gamma - {} nmax_steps'.format(
        method_name, len(evolution_happiness_all), init_epsilon, init_alpha, gamma, nmax_steps)
    
    plt.plot(x, y)
    plt.title('Happiness over time \n (smoothed over window size {})'.format(n_moving_points))
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Episode \n '+ info_parameters)
    plt.ylabel('Happiness (Smoothed)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('data/figures/{}__happiness.png'.format(method_name), format='png', dpi=500)
    plt.show()

In [32]:
def plot_comparison_evolution_happiness(evo_training__evo_avg_happiness):

    n_episodes = len(list(evo_training__evo_avg_happiness.values())[0])
    info_parameters = 'All methods - {} episodes \n {} init_epsilon - {} init_alpha - {} gamma - {} nmax_steps'.format(
        n_episodes, init_epsilon, init_alpha, gamma, nmax_steps)

    fig = plt.figure()
    cnt=0
    for method in list(evo_training__evo_avg_happiness.keys()):

        n_moving_points = int(np.ceil(len(evo_training__evo_avg_happiness[method])/100))
        y = running_mean(evo_training__evo_avg_happiness[method], n_moving_points)
        x = range(len(y))

        plt.plot(
            x, y, label=method
            , marker='', color=colors[cnt], linewidth=1, alpha=0.75
        )
        cnt += 1

    plt.title('Happiness over time \n (smoothed over window size {})'.format(n_moving_points))
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Episode \n '+ info_parameters)
    plt.ylabel('Happiness (Smoothed)')
    plt.legend(bbox_to_anchor=(0.5,-0.10), loc="lower center", 
            bbox_transform=fig.transFigure, ncol=4, fancybox=True, shadow=True, borderpad=1)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('data/figures/Comparison__happiness.png', format='png', dpi=500)
    plt.show()

# Parametrisation

In [33]:
# Defining the different parameters
init_epsilon = 1 # trade-off exploration/exploitation
init_alpha = 0.5 # learning rate
init_Q_type = 'ones'

# Episodes
n_episodes = 10000
nmax_steps = 60*24 # maximum steps per episode

# Training the learning agent - Monte-Carlo - every visit

In [34]:
method = 'MC'
method_monte_carlo = 'every_visit' # every_visit or first_visit

method_name = method + '_' + method_monte_carlo

## Initialisation

In [35]:
# Initializing the Q-matrix 
Q = init_Q(n_actions, init_Q_type)
N = init_N(n_actions)

In [36]:
# Visualisation
(render_episode, render_training) = (False, False)
n_episodes_plot = int(np.ceil(n_episodes/100))

In [37]:
# Initializing the reward
evo_training = {
    'evo_avg_reward_per_step': []
    , 'evo_n_steps': []
    , 'evo_avg_happiness': []
}

## Training

In [38]:
# Starting the SARSA learning 
for episode in tqdm(range(n_episodes)):
    
    (n_episode_steps, done) = (0, False)
    evo_episode = {
        'episode_sum_reward': 0
        , 'evolution_sum_happiness': 0
    }
    
    # Update parameters
    epsilon = get_epsilon(episode, init_epsilon)
    alpha = get_alpha(episode, init_alpha)
    
    
    # Get episode
    steps_episode = []

    state1 = env_reset()
    evo_episode['evolution_sum_happiness'] += get_happiness(state1)
    action1 = epsilon_greedy(Q, state1['state_id'], n_actions, init_epsilon)

    while (not done) and (n_episode_steps < nmax_steps):

        # Getting the next state 
        state2, reward1, done, info = env_step(state1, action1)
        evo_episode['episode_sum_reward'] += reward1
        evo_episode['evolution_sum_happiness'] += get_happiness(state2)
        
        steps_episode.append({
            'state': state1,
            'action': action1,
            'reward' : reward1})

        # Choosing the next action
        action2 = epsilon_greedy(Q, state2['state_id'], n_actions, epsilon)

        # Updating the respective values 
        state1 = state2 
        action1 = action2
        n_episode_steps += 1
    
    
    # Add discounted reward
    add_discounted_reward(steps_episode, gamma)

    
    # Update N and Q
    states_already_visited = []
    for step_episode in steps_episode:
        update_N_MC(N, step_episode, method_monte_carlo, states_already_visited)
        update_Q_MC(Q, N, step_episode, method_monte_carlo, states_already_visited)
        states_already_visited.append(step_episode['state']['state_id'])


    # At the end of learning process 
    if render_episode:
        print('Episode {0}, Score: {1}, Timesteps: {2}, Epsilon: {3}, Alpha: {4}'.format(
            episode+1, episode_reward, n_episode_steps, epsilon, alpha))
    
    evo_training['evo_avg_reward_per_step'].append(evo_episode['episode_sum_reward'] / n_episode_steps)
    evo_training['evo_n_steps'].append(n_episode_steps)
    evo_training['evo_avg_happiness'].append(evo_episode['evolution_sum_happiness'] / n_episode_steps)
    
    if ((episode+1) % n_episodes_plot == 0):
        
        with open('data/interim/{}__Q.pkl'.format(method_name), 'wb') as file:
            dill.dump(Q, file)

        with open('data/interim/{}__N.pkl'.format(method_name), 'wb') as file:
            dill.dump(N, file)

        with open('data/interim/{}__evo_training.pkl'.format(method_name), 'wb') as file:
            dill.dump(evo_training, file)

        #plot_evolution_reward(evo_training['evo_avg_reward_per_step'], method_name)
        #plot_evolution_steps(evo_training['evo_n_steps'], method_name)
        #plot_evolution_happiness(evo_training['evo_avg_happiness'], method_name)

 15%|█▌        | 1537/10000 [06:38<36:35,  3.86it/s]  


KeyboardInterrupt: 

In [ ]:
plot_evolution_reward(evo_training['evo_avg_reward_per_step'], method_name)
plot_evolution_steps(evo_training['evo_n_steps'], method_name)
plot_evolution_happiness(evo_training['evo_avg_happiness'], method_name)

# Training the learning agent - Monte-Carlo - first visit

In [ ]:
method = 'MC'
method_monte_carlo = 'first_visit' # every_visit or first_visit

method_name = method + '_' + method_monte_carlo

## Initialisation

In [ ]:
# Initializing the Q-matrix 
Q = init_Q(n_actions, init_Q_type)
N = init_N(n_actions)

In [ ]:
# Visualisation
(render_episode, render_training) = (False, False)
n_episodes_plot = int(np.ceil(n_episodes/100))

In [ ]:
# Initializing the reward
evo_training = {
    'evo_avg_reward_per_step': []
    , 'evo_n_steps': []
    , 'evo_avg_happiness': []
}

## Training

In [ ]:
# Starting the SARSA learning 
for episode in tqdm(range(n_episodes)):
    
    (n_episode_steps, done) = (0, False)
    evo_episode = {
        'episode_sum_reward': 0
        , 'evolution_sum_happiness': 0
    }
    
    # Update parameters
    epsilon = get_epsilon(episode, init_epsilon)
    alpha = get_alpha(episode, init_alpha)
    
    
    # Get episode
    steps_episode = []

    state1 = env_reset()
    evo_episode['evolution_sum_happiness'] += get_happiness(state1)
    action1 = epsilon_greedy(Q, state1['state_id'], n_actions, init_epsilon)

    while (not done) and (n_episode_steps < nmax_steps):

        # Getting the next state 
        state2, reward1, done, info = env_step(state1, action1)
        evo_episode['episode_sum_reward'] += reward1
        evo_episode['evolution_sum_happiness'] += get_happiness(state2)
        
        steps_episode.append({
            'state': state1,
            'action': action1,
            'reward' : reward1})

        # Choosing the next action
        action2 = epsilon_greedy(Q, state2['state_id'], n_actions, epsilon)

        # Updating the respective values 
        state1 = state2 
        action1 = action2
        n_episode_steps += 1
    
    
    # Add discounted reward
    add_discounted_reward(steps_episode, gamma)

    
    # Update N and Q
    states_already_visited = []
    for step_episode in steps_episode:
        update_N_MC(N, step_episode, method_monte_carlo, states_already_visited)
        update_Q_MC(Q, N, step_episode, method_monte_carlo, states_already_visited)
        states_already_visited.append(step_episode['state']['state_id'])


    # At the end of learning process 
    if render_episode:
        print('Episode {0}, Score: {1}, Timesteps: {2}, Epsilon: {3}, Alpha: {4}'.format(
            episode+1, episode_reward, n_episode_steps, epsilon, alpha))
    
    evo_training['evo_avg_reward_per_step'].append(evo_episode['episode_sum_reward'] / n_episode_steps)
    evo_training['evo_n_steps'].append(n_episode_steps)
    evo_training['evo_avg_happiness'].append(evo_episode['evolution_sum_happiness'] / n_episode_steps)
    
    if ((episode+1) % n_episodes_plot == 0):
        
        with open('data/interim/{}__Q.pkl'.format(method_name), 'wb') as file:
            dill.dump(Q, file)

        with open('data/interim/{}__N.pkl'.format(method_name), 'wb') as file:
            dill.dump(N, file)

        with open('data/interim/{}__evo_training.pkl'.format(method_name), 'wb') as file:
            dill.dump(evo_training, file)

        #plot_evolution_reward(evo_training['evo_avg_reward_per_step'], method_name)
        #plot_evolution_steps(evo_training['evo_n_steps'], method_name)
        #plot_evolution_happiness(evo_training['evo_avg_happiness'], method_name)

In [ ]:
plot_evolution_reward(evo_training['evo_avg_reward_per_step'], method_name)
plot_evolution_steps(evo_training['evo_n_steps'], method_name)
plot_evolution_happiness(evo_training['evo_avg_happiness'], method_name)

# Training the learning agent - SARSA

In [ ]:
method = 'SARSA'
method_name = method

## Initialisation

In [ ]:
# Initializing the Q-matrix 
Q = init_Q(n_actions, init_Q_type)

In [ ]:
# Visualisation
(render_episode, render_training) = (False, False)
n_episodes_plot = int(np.ceil(n_episodes/100))

In [ ]:
# Initializing the reward
evo_training = {
    'evo_avg_reward_per_step': []
    , 'evo_n_steps': []
    , 'evo_avg_happiness': []
}

## Training

In [ ]:
# Starting the SARSA learning 
for episode in tqdm(range(n_episodes)):
    
    (n_episode_steps, done) = (0, False)
    evo_episode = {
        'episode_sum_reward': 0
        , 'evolution_sum_happiness': 0
    }
    
    # Update parameters
    epsilon = get_epsilon(episode, init_epsilon)
    alpha = get_alpha(episode, init_alpha)
    
    
    # Get episode

    state1 = env_reset()
    evo_episode['evolution_sum_happiness'] += get_happiness(state1)
    action1 = epsilon_greedy(Q, state1['state_id'], n_actions, init_epsilon)

    while (not done) and (n_episode_steps < nmax_steps):

        # Getting the next state 
        state2, reward1, done, info = env_step(state1, action1)
        evo_episode['episode_sum_reward'] += reward1
        evo_episode['evolution_sum_happiness'] += get_happiness(state2)
        
        # Choosing the next action
        action2 = epsilon_greedy(Q, state2['state_id'], n_actions, epsilon)

        # Learning the Q-value
        Q = update_Q_SARSA(Q,state1['state_id'], action1, reward1, state2['state_id'], action2)
        
        # Updating the respective values 
        state1 = state2 
        action1 = action2
        n_episode_steps += 1
    

    # At the end of learning process 
    if render_episode:
        print('Episode {0}, Score: {1}, Timesteps: {2}, Epsilon: {3}, Alpha: {4}'.format(
            episode+1, episode_reward, n_episode_steps, epsilon, alpha))
    
    evo_training['evo_avg_reward_per_step'].append(evo_episode['episode_sum_reward'] / n_episode_steps)
    evo_training['evo_n_steps'].append(n_episode_steps)
    evo_training['evo_avg_happiness'].append(evo_episode['evolution_sum_happiness'] / n_episode_steps)
    
    if ((episode+1) % n_episodes_plot == 0):
        
        with open('data/interim/{}__Q.pkl'.format(method_name), 'wb') as file:
            dill.dump(Q, file)

        with open('data/interim/{}__evo_training.pkl'.format(method_name), 'wb') as file:
            dill.dump(evo_training, file)

        #plot_evolution_reward(evo_training['evo_avg_reward_per_step'], method_name)
        #plot_evolution_steps(evo_training['evo_n_steps'], method_name)
        #plot_evolution_happiness(evo_training['evo_avg_happiness'], method_name)

In [ ]:
plot_evolution_reward(evo_training['evo_avg_reward_per_step'], method_name)
plot_evolution_steps(evo_training['evo_n_steps'], method_name)
plot_evolution_happiness(evo_training['evo_avg_happiness'], method_name)

# Training the learning agent - Q-learning

In [ ]:
method = 'Q-Learning'
method_name = method

## Initialisation

In [ ]:
# Initializing the Q-matrix 
Q = init_Q(n_actions, init_Q_type)

In [ ]:
# Visualisation
(render_episode, render_training) = (False, False)
n_episodes_plot = int(np.ceil(n_episodes/100))

In [ ]:
# Initializing the reward
evo_training = {
    'evo_avg_reward_per_step': []
    , 'evo_n_steps': []
    , 'evo_avg_happiness': []
}

## Training

In [ ]:
# Starting the SARSA learning 
for episode in tqdm(range(n_episodes)):
    
    (n_episode_steps, done) = (0, False)
    evo_episode = {
        'episode_sum_reward': 0
        , 'evolution_sum_happiness': 0
    }
    
    # Update parameters
    epsilon = get_epsilon(episode, init_epsilon)
    alpha = get_alpha(episode, init_alpha)
    
    
    # Get episode

    state1 = env_reset()
    evo_episode['evolution_sum_happiness'] += get_happiness(state1)

    while (not done) and (n_episode_steps < nmax_steps):
        
        # Choose an action
        action1 = epsilon_greedy(Q, state1['state_id'], n_actions, init_epsilon)

        # Getting the next state 
        state2, reward1, done, info = env_step(state1, action1)
        evo_episode['episode_sum_reward'] += reward1
        evo_episode['evolution_sum_happiness'] += get_happiness(state2)
        

        # Q-Learning
        # Choosing the next action 
        action2 = select_best_action(Q[state2['state_id']])
        # Learning the Q-value
        Q = update_Q_Qlearning(Q, state1['state_id'], action1, reward1, state2['state_id'], action2)
        
        # Updating the respective values 
        state1 = state2
        n_episode_steps += 1
    

    # At the end of learning process 
    if render_episode:
        print('Episode {0}, Score: {1}, Timesteps: {2}, Epsilon: {3}, Alpha: {4}'.format(
            episode+1, episode_reward, n_episode_steps, epsilon, alpha))
    
    evo_training['evo_avg_reward_per_step'].append(evo_episode['episode_sum_reward'] / n_episode_steps)
    evo_training['evo_n_steps'].append(n_episode_steps)
    evo_training['evo_avg_happiness'].append(evo_episode['evolution_sum_happiness'] / n_episode_steps)
    
    if ((episode+1) % n_episodes_plot == 0):
        
        with open('data/interim/{}__Q.pkl'.format(method_name), 'wb') as file:
            dill.dump(Q, file)

        with open('data/interim/{}__evo_training.pkl'.format(method_name), 'wb') as file:
            dill.dump(evo_training, file)

        #plot_evolution_reward(evo_training['evo_avg_reward_per_step'], method_name)
        #plot_evolution_steps(evo_training['evo_n_steps'], method_name)
        #plot_evolution_happiness(evo_training['evo_avg_happiness'], method_name)

In [ ]:
plot_evolution_reward(evo_training['evo_avg_reward_per_step'], method_name)
plot_evolution_steps(evo_training['evo_n_steps'], method_name)
plot_evolution_happiness(evo_training['evo_avg_happiness'], method_name)

# Comparison

In [ ]:
list_methods = ['MC_every_visit','MC_first_visit', 'SARSA','Q-Learning']

In [ ]:
evo_training__evo_avg_reward_per_step = {}
evo_training__evo_n_steps = {}
evo_training__evo_avg_happiness = {}

for method in list_methods:
    with open("data/interim/{}__evo_training.pkl".format(method), "rb") as input_file:
        evo_training = dill.load(input_file)

    evo_training__evo_avg_reward_per_step[method] = evo_training['evo_avg_reward_per_step']
    evo_training__evo_n_steps[method] = evo_training['evo_n_steps']
    evo_training__evo_avg_happiness[method] = evo_training['evo_avg_happiness']

In [ ]:
plot_comparison_evolution_reward(evo_training__evo_avg_reward_per_step)
plot_comparison_evolution_steps(evo_training__evo_n_steps)
plot_comparison_evolution_happiness(evo_training__evo_avg_happiness)